In [4]:
import brightway2 as bw
import bw2data as bd
import dopo
from dopo import*


In [5]:
bd.projects.set_current("premise-validation-try1")
bw.bw2setup()

bio3=bw.Database('biosphere3')
ei39=bw.Database('ecoinvent 3.9.1 cutoff')
ei39SSP2=bw.Database('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27')

Biosphere database already present!!! No setup is needed


In [6]:
finder=dopo.methods.MethodFinder()

finder.find_and_create_method(criteria=['IPCC', '2013', 'GWP100'], exclude=['no LT'])
# finder.find_and_create_method(criteria=['EN15804','Cumulative', 'non-renewable' ])
# finder.find_and_create_method(criteria=['land occupation','selected'])
# finder.find_and_create_method(criteria=['EN15804','fresh water'])
method_dict=finder.get_all_methods()
method_dict

{'method_1': {'object': Brightway2 Method: IPCC 2013: climate change: global warming potential (GWP100),
  'method name': ('IPCC 2013',
   'climate change',
   'global warming potential (GWP100)'),
  'short name': 'global warming potential (GWP100)',
  'unit': 'kg CO2-Eq'}}

In [26]:
files_dict={}
files_dict['Clinker']={'yaml': r'C:\Users\fried\premise_validation\dopo\yamls\cement_steel_clinker.yaml', 
                      'yaml identifier': 'Clinker'} #yaml identifier is the name of the filter in the yaml file, in the first line.
#files_dict['Steel']= {'yaml':r'C:\Users\fried\premise_validation\dopo\yamls\steel_small.yaml',
                           # 'yaml identifier': 'Steel'}
#files_dict['Fuel']={'yaml': r'C:\Users\fried\premise_validation\dopo\yamls\fuels_small.yaml',
                   # 'yaml identifier': 'Fuels' } 
files_dict

{'Clinker': {'yaml': 'C:\\Users\\fried\\premise_validation\\dopo\\yamls\\cement_steel_clinker.yaml',
  'yaml identifier': 'Clinker'}}

In [27]:
#for comparison
premise_dict = dopo.sector_filter.process_yaml_files(files_dict, ei39SSP2)
#ecoinvent_dict = dopo.sector_filter.process_yaml_files(files_dict, ei39)


Processing Clinker with database ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27
Activities for Clinker:
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'd97ecce7cc2c13235fc2a239d2db7b1e')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '6ac3c0cfa9991b7f49bc8427de214c87')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'a1fe25988545581c0a1a3dc14c9bd703')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '9afc6ff64b5057d4e5b6c57d6a4bc584')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '464b1477852d24d39a82d30d48c15235')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '7daf2fa9e506e92babcf867fc555b319')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'b98e8c5e4e7c1dd7bd580ce67bc5e387')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'e6b12fe3a84a44f96d41173737cf46ef')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '6370363f34fe796680af3499429857f5')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '728233823577bef32e1b2e3c8ecabad2')


In [43]:
premise_dict['Clinker']['activities']

['clinker production' (kilogram, IN, None),
 'clinker production' (kilogram, ZA, None),
 'clinker production' (kilogram, PE, None),
 'clinker production' (kilogram, CH, None),
 'clinker production' (kilogram, US, None),
 'clinker production' (kilogram, CO, None),
 'clinker production' (kilogram, Europe without Switzerland, None),
 'clinker production' (kilogram, CA-QC, None),
 'clinker production' (kilogram, RoW, None),
 'clinker production' (kilogram, BR, None)]

In [44]:
type(premise_dict)

dict

In [31]:
premise_dict_1=premise_dict['Clinker']['activities'][0]

In [41]:
premise_dict_small=premise_dict['Clinker']['activities'][0:3]

In [42]:
type(premise_dict_small)

list

In [45]:
import numpy as np  # To calculate the median

database_name=ei39SSP2

# Initialize a results dictionary to store exchange data for each key
results = {}

# Iterate over each key in the premise_dict
for key, value in premise_dict.items():
    # Dictionary to store exchange data for each activity under the current key
    activities_data = {}

    try:
        # Get the list of activities for the current key
        activities_list = premise_dict[key]['activities'][:3]
    except KeyError:
        print(f"KeyError: 'activities' not found for key: {key}")
        continue

    # Check if there are activities to process
    if not activities_list:
        print(f"No activities found for key: {key}")
        continue
    
    # Iterate over each activity in the activities_list
    for activity in activities_list:
        # Initialize the counter and amounts list for this activity
        exchange_count = 0
        exchange_amounts = []

        # Retrieve the unique identifier for the activity (e.g., activity.key)
        activity_key = activity.key  # Adjust this line to match the correct attribute for key

        # Loop through all activities in the database
        for act in database_name:  # Replace 'database_name' with your actual database object
            # Loop through all exchanges in the current activity of the database
            for exc in act.exchanges():
                # Compare unique keys of exchange input and activity
                if exc.input.key == activity_key:  # Ensure we compare keys, not objects
                    exchange_count += 1
                    # Store the amount of the exchange
                    exchange_amounts.append(exc['amount'])

        # Calculate the median of the exchange amounts if there are any
        median_value = np.median(exchange_amounts) if exchange_amounts else None
        
        # Store the exchange data for the current activity
        activities_data[str(activity)] = {
            "exchange_count": exchange_count,
            "exchange_amounts": exchange_amounts,
            "median_exchange_amount": median_value,
            "activitiy_key": activity_key
        }

    # Store the activities data in the results dictionary under the current key
    results[key] = activities_data

# Print or process the results as needed
for key, activity_data in results.items():
    print(f"Results for key: {key}")
    for activity, data in activity_data.items():
        print(f"  Activity: {activity}")
        print(f"    Total Exchanges: {data['exchange_count']}")
        print(f"    Median Exchange Amount: {data['median_exchange_amount']}")
        print(f"    Exchange Amounts: {data['exchange_amounts']}")
    print("-" * 40)


Results for key: Clinker
  Activity: 'clinker production' (kilogram, IN, None)
    Total Exchanges: 5
    Median Exchange Amount: 0.955
    Exchange Amounts: [0.66, 1.0, 1.0, 0.496, 0.955]
  Activity: 'clinker production' (kilogram, ZA, None)
    Total Exchanges: 2
    Median Exchange Amount: 1.0
    Exchange Amounts: [1.0, 1.0]
  Activity: 'clinker production' (kilogram, PE, None)
    Total Exchanges: 2
    Median Exchange Amount: 1.0
    Exchange Amounts: [1.0, 1.0]
----------------------------------------


In [46]:
results

{'Clinker': {"'clinker production' (kilogram, IN, None)": {'exchange_count': 5,
   'exchange_amounts': [0.66, 1.0, 1.0, 0.496, 0.955],
   'median_exchange_amount': 0.955},
  "'clinker production' (kilogram, ZA, None)": {'exchange_count': 2,
   'exchange_amounts': [1.0, 1.0],
   'median_exchange_amount': 1.0},
  "'clinker production' (kilogram, PE, None)": {'exchange_count': 2,
   'exchange_amounts': [1.0, 1.0],
   'median_exchange_amount': 1.0}}}

In [19]:
# Debugging code to inspect the structure of main_dict
for key, value in premise_dict.items():
    print(f"Key: {key}, Type of Value: {type(value)}, Value: {value}")
    print("-" * 40)


Key: classifications, Type of Value: <class 'list'>, Value: [('ISIC rev.4 ecoinvent', '2394:Manufacture of cement, lime and plaster'), ('CPC', '37440: Portland cement, aluminous cement, slag cement and similar hydraulic cements, except in the form of clinkers')]
----------------------------------------
Key: comment, Type of Value: <class 'str'>, Value: The dataset represents an average production of cement in South Africa. The dataset describes the production of Portland cement (CEM I) and covers some relevant process steps as shown in the Figure. Portland cement production involves the chemical transformation of raw materials: calcium oxides; silica; alumina and iron oxide into various types of cementitious products, by-products and wastes. The Portland cement manufacturing process consists of four main steps: (1) Quarrying of limestone and transportation of raw materials to the processing plant. (2) Preparation of “raw meal” for pyroprocessing (3) Pyroprocessing of raw materials to p

In [8]:
key = ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '4fbb4cf48eb2fd3dd8cef6069e5223ce')

# Extract the first string from the tuple
first_string = key[0]

# Split by underscores and take the first part
database_name = first_string.split('_')[0]

print(first_string)

ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27


In [12]:
import numpy as np 

database_name=ei39SSP2
activity_key=('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27',
 'c38f699f4ad0e18953c86ede8d9a611a')

# Initialize the counter
exchange_count = 0
exchange_amounts = []

# Loop through all activities in the database
for act in database_name:
    # Loop through all exchanges in the current activity
    for exc in act.exchanges():
        # Check if the input of the exchange matches the activity key
        if exc.input == activity_key:
            exchange_count += 1
            # Store the amount of the exchange
            exchange_amounts.append(exc['amount'])

# Print the total count of exchanges
print(f"The activity '{activity_key}' is used as an exchange {exchange_count} times in the '{database_name}' database.")

# Print the amounts of the exchanges
print(f"Exchange amounts: {exchange_amounts}")

# Calculate and print the median of the exchange amounts
if exchange_amounts:
    median_amount = np.median(exchange_amounts)
    print(f"The median of the exchange amounts is: {median_amount}")
else:
    print("No exchanges found for the specified activity.")

The activity '('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'c38f699f4ad0e18953c86ede8d9a611a')' is used as an exchange 2 times in the 'Brightway2 SQLiteBackend: ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27' database.
Exchange amounts: [0.04699399422050376, 1.0]
The median of the exchange amounts is: 0.5234969971102519
